In [2]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('/Users/colwyngulliford/Documents/GitHub/lume-gpt/')

In [23]:
import tempfile
import os
import h5py
from time import time
os.getcwd()

'/Users/colwyngulliford/Documents/GitHub/lume-gpt/devel'

In [75]:
from gpt import tools, parsers, GPT

gpt_input_path = os.path.join('/Users/colwyngulliford/Documents/GitHub/lume-gpt', 'templates/cu_injector/gpt.in')
G = GPT(input_file=gpt_input_path)

G.input['lines']

map_strs = ['Map1D_E', 'Map1D_B', 'Map1D_TM', 'Map2D_E', 'Map2D_B', 'Map25D_TM']

map_defs = []

def parse_Map2D_E(line):
    
    map_def = {'type':'Map2D_E', 'gpt_line':line}
        
    line = line.replace('"','')
    line = line.replace('Map2D_E','')
    line = line.replace('(','')
    line = line.replace(')','')
    line = line.replace(';','')
    tokens = line.split(',')
    
    if(tokens[1]=='z'):
        zvar = tokens[2]
        if(tokens[2] in G.input['variables']):
            
            map_def['z']=G.input['variables'][tokens[2]]
        map_def['file']=tokens[3]
        column_names = {tokens[4]:'r', tokens[5]:'z', tokens[6]:'Er', tokens[7]:'Ez'}
        map_def['column_names'] = column_names
        if(tokens[8] in G.input['variables']):
            map_def['scale'] = G.input['variables'][tokens[8]]
    else:
        if(tokens[3] in G.input['variables']):
            map_def['z'] = G.input['variables'][tokens[3]]
            
        map_def['file']=tokens[10]
        column_names = {tokens[11]:'r', tokens[12]:'z', tokens[13]:'Er', tokens[14]:'Ez'}
        map_def['column_names'] = column_names
        
        if(tokens[15] in G.input['variables']):
            map_def['scale'] = G.input['variables'][tokens[15]]    

    return map_def

def parse_Map2D_B(line):
    
    map_def = {'type':'Map2D_B', 'gpt_line':line}
        
    line = line.replace('"','')
    line = line.replace('Map2D_B','')
    line = line.replace('(','')
    line = line.replace(')','')
    line = line.replace(';','')
    tokens = line.split(',')
    
    if(tokens[1]=='z'):
        zvar = tokens[2]
        
        if(zvar in G.input['variables']):
            map_def['z']=G.input['variables'][zvar]
        map_def['file']=tokens[3]
        column_names = {tokens[4]:'r', tokens[5]:'z', tokens[6]:'Br', tokens[7]:'Bz'}
        map_def['column_names'] = column_names
        
        if(tokens[8] in G.input['variables']):
            map_def['scale'] = G.input['variables'][tokens[8]]
    else:

        if(tokens[3] in G.input['variables']):
            map_def['z'] = G.input['variables'][tokens[3]]
            
        map_def['file']=tokens[10]
        column_names = {tokens[11]:'r', tokens[12]:'z', tokens[13]:'Br', tokens[14]:'Bz'}
        map_def['column_names'] = column_names
        
        if(tokens[15] in G.input['variables']):
            map_def['scale'] = G.input['variables'][tokens[15]]
        
    return map_def
        
for ii,line in enumerate(G.input['lines']):
    for map_str in map_strs:
        line = line.replace(' ','')
        if(line.startswith(map_str)):
            
            if(map_str=='Map2D_E'):
                map_defs.append(parse_Map2D_E(line))
            elif(map_str=='Map2D_B'):
                map_defs.append(parse_Map2D_B(line))
            
            break
            
for md in map_defs:
    print(md['type'], md['z'], md['column_names'])

Map2D_E 0.15 {'R': 'r', 'Z': 'z', 'Er': 'Er', 'Ez': 'Ez'}
Map2D_E 0.15 {'R': 'r', 'Z': 'z', 'Er': 'Er', 'Ez': 'Ez'}
Map2D_E 0.15 {'R': 'r', 'Z': 'z', 'Er': 'Er', 'Ez': 'Ez'}
Map2D_E 0.15 {'R': 'r', 'Z': 'z', 'Er': 'Er', 'Ez': 'Ez'}
Map2D_E 0.15 {'R': 'r', 'Z': 'z', 'Er': 'Er', 'Ez': 'Ez'}
Map2D_E 0.15 {'R': 'r', 'Z': 'z', 'Er': 'Er', 'Ez': 'Ez'}
Map2D_B 0.302 {'R': 'r', 'Z': 'z', 'Br': 'Br', 'Bz': 'Bz'}
Map2D_B 1.124 {'R': 'r', 'Z': 'z', 'Br': 'Br', 'Bz': 'Bz'}
